In [1]:
import datetime
import logging
import os
import pandas as pd


from utils.dialog_manipulation import prepare_dialogs_sorted_by_lang
from utils.user_input_functs import init_tool_config_arg
from utils.dialog_manipulation import add_reply_time, add_subdialogs_ids

In [3]:
# args = init_tool_config_arg()


# DIALOGS_IDS = args.dialogs_ids
# DEBUG_MODE = args.debug_mode


## change all this block to your parameters !!!!
# input dialogs_ids (it should be a list) or -1 for all
DIALOGS_IDS = [-1]
DEBUG_MODE = 0
DIALOG_PATH = os.path.join('data', 'test_dialogs')
LOGS_PATH = 'logs/project_logs.log'
PATH_TO_PREPARED_DIALOGS = os.path.join("data", "prepared_dialogs")
PATH_TO_SAVE_GENERAL_DF = os.path.join("data", "processed_dialog_files", "general_dialogs_df.csv")

# change to date range in what you want to analyse messages of user_id_get_msg - from START_DATE to END_DATE;
# format "%Y-%m-%d %H:%M:%S"
START_DATE = datetime.datetime(2020, 8, 9, 0, 0, 0)
END_DATE = datetime.datetime(2020, 8, 10, 0, 0, 0)

In [4]:
if DEBUG_MODE:
    logging.basicConfig(filename=LOGS_PATH, level=logging.DEBUG)

if os.path.isdir(DIALOG_PATH):
    if not os.path.isdir(PATH_TO_PREPARED_DIALOGS):
        os.mkdir(PATH_TO_PREPARED_DIALOGS)

    prepare_dialogs_sorted_by_lang(DIALOGS_IDS, DIALOG_PATH, PATH_TO_PREPARED_DIALOGS,
                                   START_DATE, END_DATE, "add_lang_column")

else:
    logging.error('Dialogs dir does not exist !')

dialog_ids_sorted_by_lang
{'en': [], 'ru': ['347963763'], 'ua': ['138918380']}
Loading latest local model: uk-1.1
	Loading embeddings ... 
	Loading tokenization model ...
	Loading lemmatization model ...
	Loading tagger model ...


KeyboardInterrupt: 

In [ ]:
flag_get_all = 0
if DIALOGS_IDS[0] == -1:
    DIALOGS_IDS = os.listdir(PATH_TO_PREPARED_DIALOGS)
    flag_get_all = 1

frames = []
general_df = pd.DataFrame()
general_n_subdialogs = 0
general_dialog_id = 0
len_dialogs = len(DIALOGS_IDS)

for n_dialog_id, dialog_id in enumerate(DIALOGS_IDS):
    if flag_get_all == 1:
        dialog_id = str(dialog_id)[:-4]

    data = pd.read_csv(os.path.join(PATH_TO_PREPARED_DIALOGS, f"{dialog_id}.csv"))

    add_reply_time(data)
    add_subdialogs_ids(data)

    data["dialog ID"] = n_dialog_id

    general_df = pd.concat([general_df, data])
    print(f"{n_dialog_id + 1} dialogs csv from {len_dialogs} succeeded")


general_df.rename(columns={'id': 'message_id'}, inplace=True)
general_df = general_df.drop("Unnamed: 0", axis=1)
general_df = general_df.drop("Unnamed: 0.1", axis=1)

cols = ['dialog ID', "message_id", "date", "from_id", 'to_id', 'fwd_from', 'dialog_language',
        'reply_time', 'subdialog_id', 'message', 'preprocessed_message']

general_df = general_df[cols]

general_df.to_csv(PATH_TO_SAVE_GENERAL_DF)